In [1]:
%load_ext autoreload
%autoreload 2

from langchain_aws import BedrockLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
import os
import json
from langsmith import traceable

# Load the configuration from the config.json file
with open('config.json', 'r') as config_file:
    config = json.load(config_file)

# Set the state variables
os.environ["AWS_ACCESS_KEY_ID"] = config["AWS_ACCESS_KEY_ID"]
os.environ["AWS_SECRET_ACCESS_KEY"] = config["AWS_SECRET_ACCESS_KEY"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_SECRET_KEY"]

import os


# from uuid import uuid4
# unique_id = uuid4().hex[0:8]

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "agentic-love"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = "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"  # Update to your API key

# Usage
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
openai_api_secret_key = os.getenv("OPENAI_API_KEY")



In [12]:
import openai
from langsmith.wrappers import wrap_openai
from langsmith import traceable
from langchain_openai import ChatOpenAI
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.9)

prompt = PromptTemplate(
    input_variables=["image_desc"],
    template="Generate a prompt to generate an image based on the following description, make sure to keep the prompt length under 300: {image_desc}",
)
chain = LLMChain(llm=llm, prompt=prompt)

gen_prompt = chain.run("halloween night at a haunted museum")

image_url = DallEAPIWrapper().run(gen_prompt)


In [11]:
gen_prompt

'\n\n"Imagine a spooky Halloween night at a haunted museum. The moon casts an eerie glow over the old building as you approach the entrance. You can hear the faint sound of creepy music and cackling laughter coming from within. As you enter, the dimly lit halls are filled with ghoulish artifacts and shadowy figures. Suddenly, a ghostly apparition appears before you, sending chills down your spine. Capture this bone-chilling scene in an image and share with us." '

In [13]:
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-NH5BMmrd8biLKbGRg4O7kplg/user-7laZfpcgRl5Ob4tgwKFoOC7H/img-RsKHq8Vc7ywu5qSfapdmOffU.png?st=2024-12-02T21%3A55%3A34Z&se=2024-12-02T23%3A55%3A34Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-12-02T08%3A04%3A58Z&ske=2024-12-03T08%3A04%3A58Z&sks=b&skv=2024-08-04&sig=dXqMzcinTkuUec%2B9s/XaZNybaeX6QKKR1DGqgZ7fnLw%3D'

In [4]:
from langgraph.graph import END, START, StateGraph, MessagesState
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

In [97]:
from typing import Literal
from langchain_core.messages import HumanMessage
from langchain.chat_models import ChatOpenAI # Using OpenAI for simplicity, replace with Anthropic if needed.
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
import requests
from bs4 import BeautifulSoup

# Wikipedia search tool
@tool
def wikipedia_search(query: str):
    """Searches Wikipedia for a given query and returns a summary."""
    try:
        url = f"https://en.wikipedia.org/wiki/{query.replace(' ', '_')}"

        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        content = soup.find('div', class_='mw-parser-output')
        if content:
            paragraphs = content.find_all('p')
            summary = " ".join(paragraph.text for paragraph in paragraphs[:3]).strip()
            return summary
        else:
            return "Error: Content not found on Wikipedia."
    except requests.exceptions.RequestException as e:
        return f"Error: Request failed - {e}"
    except Exception as e:
        return f"Error: An unexpected error occurred - {e}"


def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    # Check for tool calls in the *last response*, not the user's message.
    # if len(messages) > 1 and messages[-1].tool_calls:  # Check the AI's response
    #     return "tools"  # Go to tools node if the model suggests a tool call
    if len(messages) > 1 and "Error" not in messages[-1].content and len(messages[-1].content) > 100:
        return END  # Stop if a long enough, error-free summary is obtained
    else:
        return "tools" 

@traceable
def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}


In [99]:
from pydantic import BaseModel
from langchain_core.messages.ai import AIMessage


# The overall state of the graph (this is the public state shared across nodes)
class OverallState(BaseModel):
    cv : str
    summary : str
    main_skills : str
    pros : str
    cons : str

def call_model(field, model):

    response = model.invoke(field)

    if isinstance(response, AIMessage):
        response_content = response.content  # Adjust this based on the actual property
    else:
        response_content = str(response)
    
    # Return the response content as a string in OverallState
    return response_content


def make_summary(state: OverallState):

    summary_prompt = """Make a summary of the following CV, 
                        make sure to cover all skillset displayed in the candidates professional history, 
                        including leadership, engineering and others."""

    summary = call_model(summary_prompt + state.cv, model)
    output_state = OverallState(cv = state.cv, summary = summary, main_skills = "", pros = "", cons = "")

    return output_state

def get_main_skills(state: OverallState):

    get_main_skills_prompt = """ \n\n Identifying the main skills associated with this candidate. 
                            Notice that main skills, will be consistently displayed in his professional history. 
                            Also identify any rare skills that can differentiate this candidate from others.\n\n """
    leverage_summary_prompt = """ \n\n Use the summary you provided as a reference for this task as well: \n\n."""
   
    main_skills = call_model(get_main_skills_prompt + state.cv + leverage_summary_prompt + state.cv, model)
    output_state = OverallState(cv = state.cv, summary = state.summary, main_skills = main_skills, pros = "", cons = "")
    
    return output_state

def get_pros(state: OverallState):

    get_pros_prompt = "Identify the most positive points in this candidate's CV"
    leverage_summary_prompt = """ \n\n Use the summary you provided as a reference for this task as well: \n\n"""
    leverage_main_skills_prompt = """ \n\n Use the main skills you identified as a reference for this task as well: \n\n"""
    
    pros = call_model(get_pros_prompt + state.cv + leverage_summary_prompt + state.cv + leverage_main_skills_prompt + state.main_skills, model)
    output_state = OverallState(cv = state.cv, summary = state.summary, main_skills = state.main_skills, pros = pros, cons = "")
    
    return output_state

def get_cons(state: OverallState):

    get_cons_prompt = "Identify the negative points in this candidate's CV. Those points should be referenced as opportunities for improving.\n\n"
    leverage_summary_prompt = """ \n\n Use the summary you provided as a reference for this task as well: \n\n"""
    leverage_main_skills_prompt = """ \n\n Use the main skills you identified as a reference for this task as well, what is lacking? \n\n"""
    leverage_pros_prompt = """ \n\n Use the pros you identified as a reference for this task as well, what is lacking here? \n\n"""
    
    cons = call_model(get_cons_prompt + state.cv + leverage_summary_prompt + state.summary + leverage_pros_prompt + state.pros +  leverage_main_skills_prompt + state.main_skills, model)
    output_state = OverallState(cv = state.cv, summary = state.summary, main_skills = state.main_skills, pros = state.pros, cons = cons)
    
    return output_state


In [94]:

# Bind tools if necessary (Note: OpenAI's direct integration may not support 'bind_tools' if this function is specific to other libraries)
model = openai_llm  # In case you don’t need tool binding, you can directly use the model

# Modify workflow node to use OpenAI model
workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)

tool_node = ToolNode([wikipedia_search])
workflow.add_node("tools", tool_node)  # Update as necessary to ensure it aligns with the rest of your infrastructure
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

checkpointer = MemorySaver()

app = workflow.compile(checkpointer=checkpointer)

# Run the agent with OpenAI
final_state = app.invoke(
    {"messages": [HumanMessage(content="Summarize the history of Python")]},
    config={"configurable": {"thread_id": 42}}  # Ensure this configuration is supported
)

print(final_state["messages"][-1].content)

Python, a high-level programming language, was created by Guido van Rossum and first released in 1991. It was designed with an emphasis on code readability, and its syntax allows programmers to express concepts in fewer lines of code than would be possible in languages such as C++ or Java.

The language was named after the BBC show "Monty Python's Flying Circus" as Van Rossum was a fan of the show and wanted a short, unique, and slightly mysterious name for his invention.

Python 2.0 was released in 2000, which included new features like garbage collection and Unicode support. In 2008, Python 3.0 was released, which was a major revision of the language that is not completely backward-compatible. However, Python 2 was still widely used for several years, before it was officially discontinued in 2020.

Python has seen a surge in popularity in recent years, largely because of its use in a wide range of application domains, including web and internet development, scientific and numeric com

In [4]:
workflow = StateGraph()

workflow.add_node('agent', call_model)
tool_node = ToolNode([wikipedia_search])
workflow.add_node('tool', tool_node)
workflow.add_node(END)

workflow.add_edge(START, 'agent')
workflow.add_conditional_edges('agent', should_continue)
workflow.add_edge("tools", "agent")

checkpointer = MemorySaver()

app = workflow.compile(checkpointer = checkpointer)

final_state = app.invoke({"messages": [HumanMessage(content="Summarize the history of Python")]})




In [ ]:
from langchain.chat_models import ChatOpenAI # Using OpenAI for simplicity, replace with Anthropic if needed.
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode